# PICAZZO Containers

In a photonic design, it often happens that you have a photonic component that meets the functional specifications, but for some reason it does not quite fit into your design. For instance, the ports are in the wrong location, or have the wrong waveguide definition. For such common cases, IPKISS/PICAZZO provides **Containers**. These are Pcells that take another PCells and add some additional features. The commonly used PICAZZO containers are:

- **TerminatePorts**: terminate one or more ports from a structure.
- **ExtendPorts**: adds a waveguide extension to (some) ports.
- **(Auto)TransitionPorts**: adds a mode converter to ports to convert to a specific waveguide type
- **FanoutPorts**: routes ports to regular array.
- **RoutePorts**: routes ports in anothwer direction



![Different  container functions](_images/containers_ports.png)


## 0. Setup

In [ ]:
%matplotlib inline
from technologies import silicon_photonics
from ipkiss3 import all as i3
import numpy as np
import pylab as plt

In [ ]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 12, 8

To demonstrate the use of containers, we will make use of a ring resonator component:

In [ ]:
from picazzo3.filters.ring import RingRect180DropFilter
my_ring = RingRect180DropFilter()
my_ring_layout = my_ring.Layout()
my_ring_layout.visualize()

## 1. Containers

All PICAZZO containers have one thing in common: They contain another PCell, and can apply a transformation to it. The base class for containers is **_Container**, and it is not really used in its own right, because it does little more than storing another PCell.

Still, we can use it to demonstrate the basic syntax:

In [ ]:
from ipkiss3.pcell.container.container import _Container
my_ring_in_container_1 = _Container(contents = my_ring)

At the layout level the a original pcell is just place inside the container

In [ ]:
my_ring_in_container_1.Layout().visualize()

Nothing special: It looks just like the ring resonator we expect. The advantage of containers is that they are PCells as well, so they can be embedded in another container:

In [ ]:
my_ring_in_container_2 = _Container(contents = my_ring_in_container_1)
layout = my_ring_in_container_2.Layout(contents_transformation=i3.Rotation(rotation=30))

layout.visualize()

We have also applied a transformation to the placement of the original PCell.

We can see that the container referenced the other container if we look at the contents:

In [ ]:
my_ring_in_container_1.contents

The Netlist has a similar behaviour. It routes the terms of the contents to the outside terms of the container. 

In [ ]:
print my_ring.Netlist().terms
print my_ring_in_container_1.Netlist().terms

## 2. AutoTransitionPorts

In some components, there is need for a mode conversion between the original component and the routing waveguides.The  **AutoTransitionPorts** class can help with that. IPKISS has an automatic taper-selection mechanism. Based on a lookup table, it can choose the best taper for each port. 

First, let's define the Waveguide template that we want to couple to:

In [ ]:
from picazzo3.container.transition_ports import AutoTransitionPorts
from picazzo3.traces.wire_wg import WireWaveguideTemplate
wire_t = WireWaveguideTemplate()
wire_t.Layout(core_width=1.0, cladding_width=0.6 + 2 * 2.0)

Now we can transition the ring. The property ``transition_length`` sets the length of the transition used in the taper.

In [ ]:
my_ring_tapered_auto = AutoTransitionPorts(contents = my_ring,
                                           trace_template = wire_t)
my_ring_tapered_auto_layout = my_ring_tapered_auto.Layout(transition_length=20.0)
my_ring_tapered_auto_layout.visualize()

We can even use AutoTaperedPorts twice. For instance to a slot waveguide:

In [ ]:
from picazzo3.traces.slot_wg.trace import SlotWaveguideTemplate
wg_slotted = SlotWaveguideTemplate(name="my_slot_wg_template1")
wg_slotted.Layout(core_width=0.5,
            slot_width=0.12,
            cladding_width=2 * 3.0 + 0.5)

In [ ]:
my_ring_tapered2 = AutoTransitionPorts(contents = my_ring_tapered_auto,
                                      
                                      trace_template = wg_slotted)

my_ring_tapered2.Layout().visualize()

## 3. TerminatePorts

A common function with ring resonators like the one used here is that they are used as drop filters. This means one of the ports is not used. Therefore, it makes good sense to suppress this port. This can be done with the **TerminatePorts** container.

In [ ]:
from picazzo3.container.terminate_ports import TerminatePorts
my_ring_terminated = TerminatePorts(contents = my_ring,
                                    port_labels = ["out1", "in1"])
layout = my_ring_terminated.Layout()
layout.visualize()

It doesn't look any different at the layout level, because it is only a logical operation.

Note the use of **port_labels** at the pcell level while the ports are defined at the layout level and terms are defined at the netlist level. TerminatePorts will be smart enough and terminate those ports on all levels **as long as they have the same name on all the views**.


In [ ]:
print [port.name for port in layout.ports]
netlist = my_ring_terminated.Netlist()
print netlist.terms

What TerminatePorts actually does is connect a `PerfectTermination` PCell to the port. This is only a logical cell, and on the circuit level it behaves as a perfect absorber. However, this is not always the case in real life: Because an abrupt termination of a port (on the layout) can cause unwanted reflections, it is often good to have a physical termination in the form of a taper or an absorbing element. So instead of `PerfectTermination`, TerminatePorts can add your own termination PCell to any port that is terminated. In the example below, we make a broad taper to act as a stub on the terminated port.

In [ ]:
from picazzo3.apertures.basic import WireWgAperture
wire_t = WireWaveguideTemplate()
wire_t.Layout(core_width=2.0, cladding_width=6.0)
my_termination = WireWgAperture(name="my_termination", aperture_trace_template=wire_t)
layout = my_termination.Layout(transition_length=4.0)

layout.visualize()


This looks like a linear taper with somewhat angled ends. These are to avoid sharp angles in your layout that could cause errors in an automated design rule check (DRC).

Now let's add the termination taper to our ring:

In [ ]:
my_ring_terminated = TerminatePorts(contents=my_ring, port_labels=["out2",'out1'], termination=my_termination)
my_ring_terminated_layout = my_ring_terminated.Layout()
my_ring_terminated_layout.visualize()

We can see that termination also in our simulation.

In order to do a proper simulation from our terminated ring, we have to set the sensisble Caphe model parameters for all the parts. These are: 
* The directional coupler in the ring
* The termination stub. 

In [ ]:
# coupler parameters
cp = dict(cross_coupling1=1j*0.2**0.5,                       # The coupling from bus to ring and back
          straight_coupling1=(1 - 0.2)**0.5       # Coupling in the straight section
          )
# ring_params
my_ring_cm = my_ring.CircuitModel(ring_length=10.0,             # we can manually specify the ring length
                                coupler_parameters=[cp, cp])  # 2 couplers

In [ ]:
# my_termination
cm_termination = my_termination.CircuitModel(reflection=0.5**0.5)

Because we introduce a quite strong reflection in the termination, we expect to see peak splitting in our ring resonator:

In [ ]:
wavelengths=np.arange(1.5, 1.6, 0.00001)
cm_terminated_ring = my_ring_terminated.CircuitModel()
smatrix = cm_terminated_ring.get_smatrix(wavelengths=wavelengths)
plt.plot(wavelengths, np.abs(smatrix['in1', 'in2'])**2, label='through')
plt.plot(wavelengths, np.abs(smatrix['in1', 'in1'])**2, label='reflection')
plt.legend()

------------

## 4. ExtendPorts

**ExtendPorts** adds a waveguide with a certain length to the selected ports of a component. It will use the trace templates of the ports of contents (that he finds in the layout view). If you set the ports_labels to None. ExtendPorts will use all the ports in the layoutview.

In [ ]:
from picazzo3.container.extend_ports import ExtendPorts
my_ring_extended = ExtendPorts(contents = my_ring,
                               port_labels = None)

layout = my_ring_extended.Layout(extension_length=10.0)
                        
layout.visualize()

Of course you can also use the property port_labels to explicitly set the port labels you want to see extended. If you set the property 
area_layer_on=True in the Layoutview the outputwaveguides will be bundeled and will share a coverlayer.

In [ ]:
my_ring_extended = ExtendPorts(contents = my_ring,
                               port_labels = ["in1","out2"])

layout = my_ring_extended.Layout(extension_length=10.0,
                                 area_layer_on=True)
                        
layout.visualize()

---------

## 5. FanoutPorts

Often it is desirable to have all ports of a component (on one side) nicely aligned in a periodic array. This can be done with **FanoutPorts**.

Lets start simple. Here we just use a FanOut to the EAST on two ports "out1" and "in2". Just as in extend ports the property ``area_layer_on`` will add cover layer to all waveguides. 

In [ ]:
from picazzo3.container.fanout_ports import FanoutPorts

my_ring = RingRect180DropFilter()
my_ring.Layout()

from picazzo3.container.fanout_ports import FanoutPorts

my_ring_fanout = FanoutPorts(contents=my_ring,
                             port_labels=["out1","in2"])

lay = my_ring_fanout.Layout(output_direction=i3.EAST,
                            area_layer_on = True)                
lay.visualize()

If you want you can also explicitly determine the trace template to which the ports are going to be fanned out. Using the property ``auto_transition`` you can make sure that tapers are automatically addded. 

In [ ]:
from picazzo3.container.fanout_ports import FanoutPorts

my_ring = RingRect180DropFilter()
my_ring.Layout()

from picazzo3.container.fanout_ports import FanoutPorts

my_ring_fanout = FanoutPorts(contents=my_ring,
                             port_labels=["out1","in2"],
                             trace_template=wire_t,
                             auto_transition=True)

lay = my_ring_fanout.Layout(output_direction=i3.EAST,
                            area_layer_on = False)                
lay.visualize()

Similarly we can FanOut the ports "out2" and "in2" to the NORTH direction.

In [ ]:
from picazzo3.container.fanout_ports import FanoutPorts

my_ring = RingRect180DropFilter()
my_ring.Layout()

from picazzo3.container.fanout_ports import FanoutPorts

my_ring_fanout = FanoutPorts(contents=my_ring,
                             port_labels=["out2","in2"]
                             )

lay = my_ring_fanout.Layout(output_direction=i3.NORTH,
                            area_layer_on=False)

lay.visualize()

With the default settings above, the waveguides will be spaced 25um apart. However, a lot of settings can be customized.  The parameters can be best understood from the following figure:

![Parameters for FanoutPorts](_images/containers_fanout.png)

With the interact below you can play with them all. You can also play with the rotation of the ring.

In [ ]:
def make_fanout(spacing=20.0, reference_coordinate=-12.5, target_coordinate=20.0, max_s_bend_angle=45.0,bend_radius=4.5,rotation_ring=10):
    my_ring_fanout = FanoutPorts(contents=my_ring,
                                 port_labels=["out1","in2"]
                                 )

    lay = my_ring_fanout.Layout(contents_transformation=i3.Rotation(rotation = rotation_ring),#Content transformation.
                                area_layer_on=False,         # if True, adds area layer on the bundle
                                output_direction=i3.EAST,    # Output direction. 
                                spacing=spacing,            # spacing between outputs
                                reference_coordinate=reference_coordinate,  # y-coordinate (or x for NORTH and SOUTH) of first waveguide
                                target_coordinate=target_coordinate,      # x-coordinate (or y for NORTH and SOUTH) of output port
                                max_s_bend_angle=max_s_bend_angle,       # maximum angle of S-bend
                                bend_radius=bend_radius              # bend radius of waveguides
                                )                
    lay.visualize()
    

from IPython.html.widgets import interact 
interact(make_fanout,
         spacing=(10,100,5),
         reference_coordinate=(-50,50,1), 
         target_coordinate=(10,100, 1), 
         max_s_bend_angle=(1,90,0.2),
         bend_radius=(1,10,0.1),
         rotation_ring=(-180,180,1))

## 6. RoutePortsAroundCorner

A more complicated routing problem occurs when multiple ports have to be routed from one side of a component to the other. **RoutePortsAroundCorner** is designed to combine the outputs into a bundle of waveguides and route that around a corner.


In [ ]:
from picazzo3.filters.ring import RingRect180DropFilter
my_ring = RingRect180DropFilter()
from picazzo3.container.route_ports import RoutePortsAroundCorner
my_ring_routed = RoutePortsAroundCorner(contents=my_ring,
                                        port_labels=["W1", "W0"])  # ports to be routed

layout = my_ring_routed.Layout(first_step_direction=i3.DIRECTION.SOUTH,  # when rounding corner, go this direction first
                               output_direction=i3.DIRECTION.EAST,       # final output direction
                               target_coordinate=40.0, # x-coordinate where the routes align
                               first_step_spacing=8.0)      # Spacing between the waveguides.
layout.visualize()

In the example above we only routed the two west ports to the east. Let's try to route one of the east ports as well. We'll also set the **area_layer_on** parameter to False.

In [ ]:
my_ring = RingRect180DropFilter()
my_ring_routed = RoutePortsAroundCorner(contents=my_ring,
                                        port_labels=["W1", "W0", "E0"])  # ports to be routed

layout = my_ring_routed.Layout(first_step_direction=i3.DIRECTION.SOUTH,  # when rounding corner, go this direction first
                               output_direction=i3.DIRECTION.EAST,       # final output direction
                               area_layer_on=False,
                               target_coordinate=40.0, # x-coordinate where the routes align
                               first_step_spacing=8.0)      # Spacing between the waveguides.
layout.visualize()

We see 2 crossing waveguides, that's clearly not something we want. It's important to know that the order of the routes is taken from the order of the specified port_labels. In this case we want the east port to come first, so let's change this.

In [ ]:
my_ring = RingRect180DropFilter()
my_ring_routed = RoutePortsAroundCorner(contents=my_ring,
                                        port_labels=["E0", "W1", "W0"])  # ports to be routed

layout = my_ring_routed.Layout(first_step_direction=i3.DIRECTION.SOUTH,  # when rounding corner, go this direction first
                               output_direction=i3.DIRECTION.EAST,       # final output direction
                               area_layer_on=False,
                               target_coordinate=40.0, # x-coordinate where the routes align
                               first_step_spacing=8.0)      # Spacing between the waveguides.
layout.visualize()

As is the case with the other containers, **RoutePortsAroundCorner** supports Autotransitions to transition from one waveguide to another. Let's say we would want to use wider waveguides for our routes:

In [ ]:
from picazzo3.traces.wire_wg import WireWaveguideTemplate
my_ring = RingRect180DropFilter()
wtpl = WireWaveguideTemplate()
wtpl_layout = wtpl.Layout(core_width=.7)

my_ring_routed = RoutePortsAroundCorner(trace_template=wtpl,                                 
                                        contents=my_ring,
                                        port_labels=["E0", "W1"])

layout = my_ring_routed.Layout(first_step_direction=i3.DIRECTION.SOUTH,  # when rounding corner, go this direction first
                               output_direction=i3.DIRECTION.EAST,       # final output direction
                               reference_coordinate_first_step=-20.0,
                               target_coordinate=40.0,
                               area_layer_on=False,
                               first_step_spacing=8.0)      # Spacing between the waveguides.
layout.visualize()

We clearly see that the container automatically added the right transitions, all we had to do was specify which trace_template our container needs to use

## 7. RoutePortsEastWest

A common scenario for circuits, especially when designing component test suites, is to have inputs on one side, and outputs on the other. **RoutePortsEastWest** performs this fuction in a single container, while otherwise two cascaded containers would have been necessary.

For example, if we want a single input in the ring, while all other ports act as output, we could use the following container:

In [ ]:
from picazzo3.filters.ring import RingRect180DropFilter
from picazzo3.container.route_ports import RoutePortsEastWest

my_ring = RingRect180DropFilter()

my_ring_ew = RoutePortsEastWest(contents=my_ring,
                                east_port_labels=["E0", "E1", "W1"],   # ports to be routed eastwards
                                west_port_labels=["W0"])             # ports to be routed westward
layout = my_ring_ew.Layout()
layout.visualize()

The east & west ports are calculated from the transformed content. It's important to remember that east ports are all ports that have an angle between -45 and 45 degrees. Likewise west ports are ports with an angle between 135 and 225 degrees. This is illustrated with following interactive example:

In [ ]:
def make_routed_ring(rotation=0.0):
    my_ring = RingRect180DropFilter()
    my_ring_ew = RoutePortsEastWest(contents=my_ring)             # ports to be routed westward
    layout = my_ring_ew.Layout(contents_transformation=i3.Rotation(rotation=rotation),
                               area_layer_on=False)

    return layout.visualize()

In [ ]:
from IPython.html.widgets import interact 
interact(make_routed_ring,
         rotation=(0.0, 360.0, 5.0))
         

You'll notice that when you choose an angle that is in between 45 and 135 degrees, you don't see any routes. This happens because there are no 'east' or 'west' ports anymore, so the container doesn't know which ports it needs to route. 

Just like RoutePortsAroundCorner, RoutePortsEastWest supports auto transitions to make transitions from 1 waveguide type to another.

In [ ]:
from picazzo3.traces.wire_wg import WireWaveguideTemplate
wtpl = WireWaveguideTemplate()
wtpl_layout = wtpl.Layout(core_width=.7)
my_ring = RingRect180DropFilter()
my_ring_routed = RoutePortsEastWest(trace_template=wtpl,                                 
                                    contents=my_ring)

layout = my_ring_routed.Layout(area_layer_on=False)
layout.visualize()